# imports and db connection setup

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import kmeans_plusplus
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
import plotly.express as px

In [ ]:
# get credentials from environment variables
user = os.getenv('PGUSER')
password = os.getenv('PGPASSWORD')
host = os.getenv('PGHOST')
port = os.getenv('PGPORT')
database = os.getenv('PGDATABASE')

# configure connection to postgres
engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database))

# open a connect
db_conn = engine.connect()

# get data from db

In [ ]:
df = pd.read_sql("select * from modelling.acdhs_program_participation_and_evictions8;", db_conn)
#df = db_conn.execute(sql_template)

In [ ]:
df.shape


In [ ]:
df

# cluster the different programs offered by DHS

features to include:
- all starting with "p_"
- expect homelessness

In [ ]:
features = [col for col in df.columns if "p_" in col and col not in ["p_263", "p_29", "p_32", "p_33"]]

In [ ]:
print(list(df.columns))

In [ ]:
X = df[features]

In [ ]:
X

In [ ]:
X.isnull().sum() # no missing values

## PCA

In [ ]:
#PCA with one/two/three principal components
pca_1d = PCA(n_components=1)
pca_2d = PCA(n_components=2)
pca_3d = PCA(n_components=3)

In [ ]:
n_comps = []
cumulative_variance = []
#for n_comp in range(1, X.shape[1]):
#for n_comp in range(1, 30):
for n_comp in [1,2,5,10,15,25, 50, 75, 100, 125, 150]:
    n_comps.append(n_comp)
    pca = PCA(n_components=n_comp)
    pca_result = pca.fit_transform(X)
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    print('Cumulative variance explained by {} principal components: {:.2%}'.format(n_comp, np.sum(pca.explained_variance_ratio_)))
    cumulative_variance.append(np.sum(pca.explained_variance_ratio_))

In [ ]:
plt.plot(n_comps, cumulative_variance)
plt.xlabel("number of components")
plt.ylabel("Cumulative variance explained by components")

In [ ]:
# Finding important features with the help of PCA
pca_2d = PCA(n_components=2)
pca_result = pca_2d.fit_transform(X)
dataset_pca = pd.DataFrame(abs(pca_2d.components_), columns=X.columns, index=['PC_1', 'PC_2'])
print('\n\n', dataset_pca)

## clustering

### Hyperparameter tuning using the elbow method

choose number of clusters

In [ ]:
# calculate distortion for a range of number of cluster
distortions = []
#nr_of_clusters = [1,2,3,4,5]
nr_of_clusters = [1,2,3,4,5,7,9,11,15,20,25,30,35,40,50,75]
for i in nr_of_clusters:
    print(i, "clusters")
    km = KMeans(
        n_clusters=i, init='k-means++',
        n_init=10, max_iter=100,
        tol=1e-04, random_state=0
    )
    km.fit(X)
    distortions.append(km.inertia_)

# plot
plt.plot(nr_of_clusters, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

## run kmeans

In [ ]:
#optimum_num_clusters = best_grid['n_clusters']
optimum_num_clusters = 7

# fitting KMeans
kmeans = KMeans(n_clusters=optimum_num_clusters)
kmeans.fit(X)
centroids = kmeans.cluster_centers_
centroids_pca = pca_2d.transform(centroids)

# visualizations

In [ ]:
def visualizing_results(pca_result, label, centroids_pca):
    """ Visualizing the clusters
    :param pca_result: PCA applied data
    :param label: K Means labels
    :param centroids_pca: PCA format K Means centroids
    """
    x = pca_result[:, 0]
    y = pca_result[:, 1]

    plt.scatter(x, y, c=label, alpha=0.1, s=10)  # plot different colors per cluster
    plt.title('DHS program participation')
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')

    plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], marker='X', s=200, linewidths=1.5,
                color='red', edgecolors="black", lw=1.5)

    plt.show()

In [ ]:
centroids_pca

In [ ]:
visualizing_results(pca_result, kmeans.labels_, centroids_pca)

## cluster centroids

In [ ]:
# get all program names
program_names = pd.read_sql("select program_key, program_name from lookup.program_feed pf ;", db_conn)
program_names

In [ ]:
print("kmeans has seen", kmeans.n_features_in_, "during fit.")
centroid_weights = zip(range(optimum_num_clusters), centroids)
for cluster_nr, c in centroid_weights:
    print("Cluster:", cluster_nr)
    #sorted_zip = sorted(cluster_and_programs, key = lambda x: x[1])
    cluster_and_programs_sorted = sorted(list(zip(kmeans.feature_names_in_,c)), key = lambda x: x[1], reverse=True)
    counter = 0
    for feature, weights in cluster_and_programs_sorted:
        #program_name = program_names[program_names["program_key"]==feature.lstrip("p_")]["program_name"][0]
        #print(program_names[program_names["program_key"]==1]["program_name"][0])
        #print("a", feature.lstrip("p_"))
        #print(program_names[program_names["program_key"]==int(feature.lstrip("p_"))])
        program_name = list(program_names[program_names["program_key"]==int(feature.lstrip("p_"))]["program_name"])[0]
        print("\t", program_name, feature, weights)
        counter += 1
        if counter >30:
            break

In [ ]:
centroids

In [ ]:
#Find which cluster each data-point belongs to
clusters = kmeans.predict(X)
#Add the cluster vector to our DataFrame, X
#X["Cluster"] = clusters

## analyse the clusters and check share of homeless individuals

In [ ]:
X_clustered = X.copy(deep=True)
X_clustered["cluster"] = kmeans.predict(X)
X_clustered["cluster_dist"] = [np.min(i) for i in kmeans.transform(X)]
X_clustered["homeless"] = df['p_263'].replace([0,1],["No", "Yes"])

In [ ]:
sum(X_clustered[X_clustered["cluster"]==0].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==0].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 0 (n=" + str(X_clustered[X_clustered["cluster"]==0].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==0) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==0].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==1].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 1 (n=" + str(X_clustered[X_clustered["cluster"]==1].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==1) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==1].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==2].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 2 (n=" + str(X_clustered[X_clustered["cluster"]==2].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==2) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==2].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==3].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 3 (n=" + str(X_clustered[X_clustered["cluster"]==3].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==3) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==3].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==4].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 4 (n=" + str(X_clustered[X_clustered["cluster"]==4].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==4) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==4].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==5].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 5 (n=" + str(X_clustered[X_clustered["cluster"]==5].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==5) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==5].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

In [ ]:
programs_grouped_by_hl = X_clustered[X_clustered["cluster"]==6].drop(columns=['cluster', 'cluster_dist']).groupby("homeless").sum().reset_index()
px.bar(programs_grouped_by_hl.melt(id_vars=["homeless"]),x="variable", y="value", color="homeless", title = "Cluster 6 (n=" + str(X_clustered[X_clustered["cluster"]==6].shape[0]) + " / n_homeless=" + str(X_clustered[(X_clustered["cluster"]==6) & (X_clustered["homeless"]=="Yes")].shape[0]) + " / n_program_participation = " + str(sum(X_clustered[X_clustered["cluster"]==6].drop(columns=['cluster', 'cluster_dist', 'homeless']).sum())) + ")")

# individuals closest to each cluster centroid

In [ ]:
for cluster_nr in range(optimum_num_clusters):
    print("Cluster:", cluster_nr)
    closest_individuals = X_clustered[X_clustered["cluster"] == cluster_nr]["cluster_dist"].nsmallest(20)
    closest_individuals_homeless = X_clustered[(X_clustered["cluster"] == cluster_nr) & (X_clustered["homeless"] == "Yes")]["cluster_dist"].nsmallest(20)
    for i, value in closest_individuals.iteritems():
        print("\t\t client_hash:", df.loc[i, "client_hash"], "cluster_distance:", value)
    print("\t closest homeless individuals:")
    for i, value in closest_individuals_homeless.iteritems():
        print("\t\t client_hash:", df.loc[i, "client_hash"], "cluster_distance:", value)